텐서플로우를 사용하기 위해 임포트하기

In [ ]:
import tesorflow as tf

x 에 특정한 값이 주어진 것은 아님
이는 플레이스 홀더(텐서플로우에서 연산을 실행할 때 값을 입력할 자리.
784차원의 벡터로 변형된 mnist 이미지의 데이터를 넣으려고 함. ( [None, 784] 의 형태, 부동소수점으로 이루어진 2차원 텐서로 표현함. 추가로 None 은 해당 차원의 길이가 어떤 길이든지 될수 있음을 뜻함. )

In [ ]:
x = tf.placeholder(tf.float32, [None, 784])

tf.Variable에 Variavle의 초기 값을 넘겨줌으로써 이 Variable 들을 생성합니다 : 여기서는 w와 b 둘 다 0으로 이루어진 텐서로 초기화 함.
w 가 [784, 10]의 형태를 갖는 것에 주목!. reason = w dp 784차원의 이미지 벡터를 곱해서 각 클래스에 대한 증거 값을 나타내는 10차원 벡터를 얻고자하기 때문, b는 그 10차원 벡터에 더하기 위해 [10] 의 형태를 갖춤.

In [ ]:
w = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros({10}))

모델구현임, 우선, tf.matmul(x, w) 로 x랑 w를 곱함. x가 여러 입력값을 갖는 2차원 텐서인 경우에도 대응하기 위한 작은 트릭. 그 다음엔 b를 더하고 마지막으로 tf.nn.softmax를 적용함.
텐서플로우는 다양한 종류의 수치 연산을 표현할 수 있는 유연한 방법이다. 그리고 한 번 작성한 모델은 여러 기기에서 실행할 수 있다.

In [ ]:
y = tf.nn.softmax(tf.matmul(x, w) + b)

모델의 손실을 정의하기 위해 자주 사용되는 함수 크로스 엔트로피 = 우리의 예측이 실제 값을 설명하기에 얼마나 비효율적인지를 측정하는 것. 크로스 엔트로피를 구현하기 위해서는 올바른 답을 넣기 위한 새로운 플레이스홀더를 추가하는 것 부터 시작해야 함.

In [ ]:
y_ = tf.placeholder(tf.float32, [None, 10])

크로스 엔트로피 구현. 우선, tf log는 y의 각 원소의 로그 값을 계산함. 그 다음, y_ 의 각 원소를 tf.log9y)의 해당하는 원소들과 곱함. 그리고 tf.reduce_sum 으로 y의 2번쨰 차원( reduction_indices=[1] 이라는 파라미터가 주어졌으므로) 의 원소들을 함함. 마지막으로, tf.reduce_mean으로 batch(배치)의 모든 예시에 대한 평균을 계산함. 근데 이거 쓰지말래 헿

In [ ]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

텐서플로우에게 학습 비율 0.5로 경사 하강법(gradent descent 알고리즘)을 적용하여 크로스 엔트로피를 최소화하도록 지시. 경사하강법이란 텐서플로우가 각각의 변수를 비용을 줄이는 방향으로 조금씩 이동시키는 매우 단순한 방법. but 텐서플로우는 다른 최적화 알고리즘 여러개 제공함. 여기서 텐서플로우가 실제로 뒤에서 하는 일은, 역전파와 경사하강이라는 새로운 작업을 당신의 그래프에 추가하는 것임. 이제 텐서플로우가 실행되면 비용을 감소시키기 위해 변수들을 살짝 수정하는 경사 하강 학습 작업 한 번을 돌려줄거임. 이제 우리의 모델은 학습할 준비됨

In [ ]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

학습을 실행시키기 전에 마지막으로, 우리가 작성한 변수들을 초기화 하는 작업을 추가해야 함.

In [ ]:
init = tf.global_variables_initializer()

이제 session에서 모델을 실행시키고, 변수들을 초기화 하는 작업을 실행시킬 수 있음.

In [ ]:
sess = tf.Session()
sess.run(init)

학습을 시킨다. 크크. 여기선 학습을 1000번 시킴. 반복되는 루프의 각 단계마다 우리는 학습 데이터셋에서 무작위로 선택된 100개의 데이터로 구성된 batch를 가져옴. 그 다음엔 플레이스홀더의 자리에 데이터를 넣을 수 있도록 train_step을 실행하여 배치 데이터를 넘김. 무작위 데이터의 작은 배치를 사용하는 방법을 확률적 학습(stochastic training)이라고 부름. 여기서는 확률적 경사 하강법임. 

In [ ]:
for i in range(1000):
  batch_xs, batch_ys = mnist.train.next.batch(100)
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

모델 평가하기,
우선 모델이 라벨을 올바르게 예측했는지 확인해보기 tf.argmax는 텐서 안에서 특정 축을 따라 가장 큰 값의 인덱스를 찾기에 매우 유용한 함수입니다. 예를 들면, tf.argmax(y,1)는 모델이 생각하기에 각 데이터에 가장 적합하다고 판단한(가장 증거값이 큰)라벨이며 , tf.argmax(y_,1)는 실제 라벨입니다. 우리는 tf.equal을 사용하여 우리의 예측이 맞았는지 확인할 수 있습니다. 이렇게 하면 부울 값으로 이루어진 리스트를 얻게 됩니다. 

In [ ]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

얼마나 많이 맞았는지 판단하려면, 이 값을 부동소수점 값으로 변환한 후 평균을 계산하면 됩니다. 예를 들면, [True, False, True, True] 는 [1,0,1,1] 로 환산할 수 있고, 이 값의 평균을 계사하면 0.75가 됩니다.

In [ ]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

마지막으로, 테스트 데이터를 대상으로 정확도 계산

In [ ]:
print(sess.run(accuracy, feed_dict={x:mnist.testimages, y_: mnist.test.labels}))